In [1]:
# Imports
import pandas as pd
import numpy as np
import psycopg2
import tqdm
from time import gmtime, strftime

#### POSTGRESQL

In [2]:
con = psycopg2.connect(dbname ='mimic', user='postgres', password="*2465RMLbru", host="/var/run/postgresql")
cur = con.cursor()

### Patients table

In [3]:
pat_query = '''SELECT * FROM mimiciii.patients;'''
mimic_patients_df = pd.read_sql_query(pat_query, con)

### Admission table

In [4]:
adm_query = '''SELECT * FROM mimiciii.admissions;'''
mimic_admission_df = pd.read_sql_query(adm_query, con)

#### Merge both tables and calculate true age

In [5]:
# Merge patient and admission tables
mimic_patients_admission_df = pd.merge(mimic_admission_df, mimic_patients_df, on = 'subject_id' , how = 'inner')

In [6]:
# ADMITTIME and DOB objects to datetime
mimic_patients_admission_df["admittime"] = pd.to_datetime(mimic_patients_admission_df["admittime"], format='%Y%m%d %H:%M:%S')
mimic_patients_admission_df["dob"] = pd.to_datetime(mimic_patients_admission_df["dob"], format='%Y%m%d %H:%M:%S')

# True age
mimic_patients_admission_df["ages"] = mimic_patients_admission_df["admittime"].sub(mimic_patients_admission_df["dob"]).dt.days/365.242

# Adult patients only
mimic_patients_admission_df = mimic_patients_admission_df[(mimic_patients_admission_df['ages'] >= 18)]

### ICD-9 Codes table

In [7]:
diagnoses_query = '''SELECT * FROM mimiciii.diagnoses_icd;'''
mimic_diagnoses_df = pd.read_sql_query(diagnoses_query, con)

In [8]:
mimic_patients_diagnoses_df = pd.merge(mimic_patients_admission_df, mimic_diagnoses_df, on = ['subject_id', 'hadm_id'], how = 'inner')

### ICD-9 Descriptions table

In [9]:
diagnoses_descriptions_query = '''SELECT * FROM mimiciii.d_icd_diagnoses;'''
mimic_diagnoses_descriptions_df = pd.read_sql_query(diagnoses_descriptions_query, con)

## Occurrence and Mortality

### By ethnicity

In [10]:
# function by wboeg
def normalize_ethnicity(ethnicity):
    if 'BLACK' in ethnicity:
        return 'BLACK'
    if 'WHITE' in ethnicity:
        return 'WHITE'
    if 'ASIAN' in ethnicity:
        return 'ASIAN'
    if 'LATINO' in ethnicity:
        return 'LATINO'
    if 'HISPANIC' in ethnicity:
        return 'LATINO'
    if 'SOUTH AMERICAN' in ethnicity:
        return 'LATINO'
    if 'CARIBBEAN ISLAND' in ethnicity:
        return 'OTHER'
    if 'AMERICAN INDIAN' in ethnicity:
        return 'OTHER'    
    if 'MIDDLE EASTERN' in ethnicity:
        return 'OTHER'
    if 'MULTI RACE ETHNICITY' in ethnicity:
        return 'OTHER'
    if 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' in ethnicity:
        return 'OTHER'
    if 'PORTUGUESE' in ethnicity:
        return 'OTHER'       
    return 'UNKNOWN'

In [11]:
mimic_patients_diagnoses_df['ethnicity'] = mimic_patients_diagnoses_df['ethnicity'].apply(normalize_ethnicity)

In [44]:
# Patients who stayed more than 6 hours
inds_at_least_6hrs = (mimic_patients_diagnoses_df['dischtime'] - mimic_patients_diagnoses_df['admittime']) > pd.Timedelta(hours=6)
mimic_patients_diagnoses_df = mimic_patients_diagnoses_df.loc[inds_at_least_6hrs]

los = mimic_patients_diagnoses_df['dischtime'] - mimic_patients_diagnoses_df['admittime']
mimic_patients_diagnoses_df['los'] = los.apply(lambda t:t.seconds/3600.)

#### Unique patient with multiple rows with same ICD9

In [46]:
eth_mortality_df = mimic_patients_diagnoses_df.groupby(['icd9_code', 'ethnicity', 'hospital_expire_flag']).size().unstack()
eth_mortality_df = eth_mortality_df.reset_index()
eth_mortality_df.columns.names = [None]
eth_mortality_df.columns = ['icd9_code', 'ethnicity', 'alive', 'dead']
eth_mortality_df.insert(4, 'total', 'NULL')
eth_mortality_df = eth_mortality_df.fillna(0)

# Compute alive, dead and total
for index, row in eth_mortality_df.iterrows():
    eth_mortality_df.at[index, 'total'] = row['alive'] + row['dead']

In [47]:
# We are not considering MULTI RACE ETHNICITY, UNKNOWN or OTHER 
eth_mortality_df = eth_mortality_df[(eth_mortality_df['ethnicity'] != 'OTHER') & (eth_mortality_df['ethnicity'] != 'UNKNOWN')]

#### Transplanted patients

In [48]:
# merge mortality with descriptions from each ICD
eth_mortality_df = eth_mortality_df.merge(mimic_diagnoses_descriptions_df, left_on='icd9_code', right_on='icd9_code')

# Search only for patients with transplant description
eth_mortality_df = eth_mortality_df[eth_mortality_df['long_title'].str.lower().str.contains('transplant')]

In [49]:
# 1. Mortality significance: only +1 patients dead
eth_mortality_df = eth_mortality_df.loc[eth_mortality_df['dead'] > 1].copy()

# 2. Cases with black and white patients
eth_mortality_df =eth_mortality_df[eth_mortality_df['ethnicity'].isin(['WHITE', 'BLACK'])]

# 3. Remove ICD9 codes with only ONE ETHNICITY
for index, row in eth_mortality_df.iterrows():
    rows = eth_mortality_df.loc[eth_mortality_df['icd9_code'] == row['icd9_code']]
    if (len(rows) == 1):
        eth_mortality_df.drop(rows.index, inplace=True)

In [50]:
eth_mortality_df = eth_mortality_df.drop(['row_id', 'short_title'], axis=1)

In [51]:
pd.set_option("display.max_colwidth", 500, 'display.max_rows', 100)
eth_mortality_df

,icd9_code,ethnicity,alive,dead,total,long_title
11136,99681,BLACK,99.0,5.0,104,Complications of transplanted kidney
11138,99681,WHITE,313.0,35.0,348,Complications of transplanted kidney
11140,99682,BLACK,14.0,4.0,18,Complications of transplanted liver
11142,99682,WHITE,112.0,20.0,132,Complications of transplanted liver
11146,99685,BLACK,7.0,2.0,9,Complications of transplanted bone marrow
11148,99685,WHITE,65.0,50.0,115,Complications of transplanted bone marrow
11636,E8780,BLACK,61.0,6.0,67,"Surgical operation with transplant of whole organ causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation"
11638,E8780,WHITE,262.0,29.0,291,"Surgical operation with transplant of whole organ causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation"
12669,V420,BLACK,56.0,5.0,61,Kidney replaced by transplant
12671,V420,WHITE,225.0,16.0,241,Kidney replaced by transplant


##### HOSPITAL ADMISSIONS FOR EACH ICD9

In [52]:
icd9_list = set(eth_mortality_df['icd9_code'])

#icd9_hadm_dict = {}
hadm_ids_list = []

# Only white and black patients and corresponding admissions
for icd9 in icd9_list:
    transplanted_patients_df = mimic_patients_diagnoses_df.loc[(mimic_patients_diagnoses_df["icd9_code"] == icd9)]
    hadm_ids = transplanted_patients_df['hadm_id']
    for hadm_id in hadm_ids:
        #icd9_hadm_dict.setdefault(icd9, []).append(hadm_id)
        hadm_ids_list.append(hadm_id)

#### Note Events

In [53]:
# Read Notes
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

notes_query = 'select distinct n.subject_id, n.hadm_id, n.category, n.text, n.chartdate, n.charttime \
from mimiciii.noteevents n  \
where iserror IS NULL and \
hadm_id in (' + ','.join(map(str, hadm_ids_list)) + ')'

notes = pd.read_sql_query(notes_query, con)
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2019-11-07 20:33:06
2019-11-07 20:33:10


In [56]:
# When patient did the transplant ?
transplant_ids = []
for hadm_id, rows in tqdm.tqdm(notes.groupby('hadm_id')):
    for text in rows.text.values:
        if 'surgical intensive care unit' in text.lower():
            transplant_ids.append(hadm_id)

100%|██████████| 1507/1507 [00:00<00:00, 2884.57it/s]


In [58]:
transplant_ids

[103445,
 103487,
 106247,
 106247,
 108592,
 111519,
 118838,
 122395,
 123898,
 125281,
 126354,
 127919,
 127920,
 128814,
 129093,
 132912,
 133228,
 140642,
 142286,
 143970,
 144161,
 144409,
 145466,
 146403,
 147511,
 148418,
 149687,
 152440,
 158129,
 159694,
 161120,
 168786,
 169761,
 170045,
 171544,
 172927,
 176969,
 176969,
 182479,
 185873,
 186508,
 187880,
 187982,
 189501,
 194686]

In [69]:
pd.set_option("display.max_colwidth", 10000)
mimic_patients_diagnoses_df.loc[(mimic_patients_diagnoses_df['subject_id'] == 15160)]

,row_id_x,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,dob,dod,dod_hosp,dod_ssn,expire_flag,ages,row_id,seq_num,icd9_code,los
140745,18533,15160,128814,2106-04-26 23:08:00,2106-05-03 14:13:00,NaT,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2059-05-09,NaT,NaT,NaT,0,46.963383,169144,1.0,25061,15.083333
140746,18533,15160,128814,2106-04-26 23:08:00,2106-05-03 14:13:00,NaT,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2059-05-09,NaT,NaT,NaT,0,46.963383,169145,2.0,3572,15.083333
140747,18533,15160,128814,2106-04-26 23:08:00,2106-05-03 14:13:00,NaT,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2059-05-09,NaT,NaT,NaT,0,46.963383,169146,3.0,25051,15.083333
140748,18533,15160,128814,2106-04-26 23:08:00,2106-05-03 14:13:00,NaT,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2059-05-09,NaT,NaT,NaT,0,46.963383,169147,4.0,36201,15.083333
140749,18533,15160,128814,2106-04-26 23:08:00,2106-05-03 14:13:00,NaT,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,2059-05-09,NaT,NaT,NaT,0,46.963383,169148,5.0,V420,15.083333
140750,18534,15160,121104,2106-07-15 21:18:00,2106-08-16 15:46:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,2059-05-09,NaT,NaT,NaT,0,47.182416,169135,1.0,9974,18.466667
140751,18534,15160,121104,2106-07-15 21:18:00,2106-08-16 15:46:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,2059-05-09,NaT,NaT,NaT,0,47.182416,169136,2.0,5609,18.466667
140752,18534,15160,121104,2106-07-15 21:18:00,2106-08-16 15:46:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,2059-05-09,NaT,NaT,NaT,0,47.182416,169137,3.0,431,18.466667
140753,18534,15160,121104,2106-07-15 21:18:00,2106-08-16 15:46:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,2059-05-09,NaT,NaT,NaT,0,47.182416,169138,4.0,99859,18.466667
140754,18534,15160,121104,2106-07-15 21:18:00,2106-08-16 15:46:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,...,2059-05-09,NaT,NaT,NaT,0,47.182416,169139,5.0,2639,18.466667


In [71]:
notes.loc[(notes['hadm_id'] == 128814) & (notes['category'] == 'Discharge summary')]

subject_id  hadm_id           category  \
17211       15160   128814  Discharge summary   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
# make sure each hadm_id has only died once
assert len(eol_cohort) == len(set(eol_cohort['hadm_id'].values))
print 'eol subjects:', len(set(eol_cohort['hadm_id'].values))

#### Cancer

In [ ]:
searchfor = ['neoplasm', 'neoplasms', 'sarcoma', 'carcinoma']
mortality_cancer_df = eth_mortality_merged_df[eth_mortality_merged_df['long_title'].str.lower().str.contains('|'.join(searchfor))]

In [ ]:
# 1. Mortality significance: only +1 patients dead
cancer_df = mortality_cancer_df.loc[mortality_cancer_df['dead'] > 1].copy()

# 2. Remove ICD9 codes with only ONE ETHNICITY
for index, row in cancer_df.iterrows():
  rows = cancer_df.loc[cancer_df['icd9_code'] == row['icd9_code']]
  if (len(rows) == 1):
    cancer_df.drop(rows.index, inplace=True)

In [ ]:
cancer_df = cancer_df.drop(['row_id', 'short_title'], axis=1)

In [ ]:
# Breast cancer
cancer_df[cancer_df['icd9_code'].str.lower().str.contains(r'^174.*')]

In [ ]:
#  Prostate cancer
cancer_df[cancer_df['icd9_code'].str.lower().str.contains(r'^185.*')]

In [ ]:
pd.set_option("display.max_colwidth", 500, 'display.max_rows', 200)

# Malignant Neoplasm Of Other And Unspecified Sites
cancer_df[cancer_df['icd9_code'].str.lower().str.contains(r'^19.*')]